<a href="https://colab.research.google.com/github/Vakhranev/Gramota/blob/main/%D0%A1%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D0%B8%20%D0%B2%20XML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import xml.etree.ElementTree as ET
import re

# Список сокращений
abbreviations = set([
    "адыг.", "америк.", "англ.", "арабск.", "арамейск.", "ассир.", "атт.", "баскск.", "библ.",
    "болг.", "букв.", "валлийск.", "венг.", "вост.", "в-т", "галльск.", "гаек.", "гаэльск.", "герм.",
    "гот.", "греч.", "груз.", "дагест.", "дат.", "диал.", "др.", "евр.", "егип.", "ж.", "зап.", "им.",
    "ирл.", "исл.", "исп.", "ит.", "кавк.", "карфагенск.", "кельт.", "коптск.", "корейск.", "коре.",
    "кумык.", "ласк.", "лат.", "лит.", "м.", "марийск.", "н.", "нар.", "нем.", "нидерл.", "нов.",
    "норв.", "норм.", "орф.", "осет.", "перс.", "пожелат.", "половецк.", "польск.", "порт.", "прил.",
    "прованс.", "разг.", "римск.", "румынск.", "рус.", "савойск.", "санскр.", "сербск.", "сирийск.",
    "сканд.", "слав.", "см.", "сокр.", "ср.", "стар.", "тат.", "традиц.", "тур.", "тюрк.", "укр.",
    "уменьш.", "усеч.", "ф.", "финск.", "флам.", "фракийск.", "фр.", "фриз.", "халдейск.", "церк.",
    "цыг.", "чешек.", "швед.", "шотл.", "эльз.", "эолийск.", "др.-евр."
])

# Функция обработки XML
def process_xml(input_file, output_file):
    tree = ET.parse(input_file)
    root = tree.getroot()

    page_number = 1
    new_pages = []

    for page in root.findall('.//page'):
        new_page = ET.Element('page', {'number': str(page_number)})
        page_number += 1

        paragraphs = [p.text.strip() for p in page.findall('.//paragraph') if p.text]
        text = " ".join(paragraphs)

        if text == "ЖЕНСКИЕ ИМЕНА":
            new_page.append(ET.Element('female_names'))
            new_page[-1].text = text
            new_pages.append(new_page)
            continue
        elif text == "МУЖСКИЕ ИМЕНА":
            new_page.append(ET.Element('male_names'))
            new_page[-1].text = text
            new_pages.append(new_page)
            continue

        words = text.split()
        i = 0

        while i < len(words):
            if words[i][0].isupper():
                word = words[i]
                i += 1

                definition_words = []
                open_square_brackets = 0
                open_parentheses = 0

                while i < len(words):
                    definition_words.append(words[i])

                    # Проверка на открытые скобки
                    open_square_brackets += words[i].count('[')
                    open_square_brackets -= words[i].count(']')

                    # Если встречаем точку, проверяем завершение
                    if words[i][-1] == '.':
                        # Проверка на сокращения
                        if words[i] not in abbreviations and open_square_brackets == 0 and open_parentheses == 0:
                            # Проверка следующего слова
                            if i + 1 < len(words) and words[i + 1][0].isupper():
                                # Если следующее слово с большой буквы, то завершение
                                break
                    i += 1

                # Собираем определение и добавляем его в запись
                definition = " ".join(definition_words).strip()

                entry = ET.Element('entry')
                word_elem = ET.SubElement(entry, 'word')
                word_elem.text = word.rstrip(',')
                def_elem = ET.SubElement(entry, 'definition')
                def_elem.text = definition
                new_page.append(entry)

            i += 1

        new_pages.append(new_page)

    root.clear()
    for new_page in new_pages:
        root.append(new_page)

    tree.write(output_file, encoding="utf-8", xml_declaration=True)

# Входной и выходной файлы
input_file = 'Superanskaya_Slovar_russkikh_lichnykh_imyon.xml'
output_file = 'processed_Slovar.xml'

# Запуск обработки
process_xml(input_file, output_file)